In [354]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from datetime import datetime, timedelta
import time

In [364]:
headers = {
'X-Requested-With':'XMLHttpRequest',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36'
}

payload = {
    "sCity":"", #出發地
    "sLine":",", #線別,副總別
    "sLine1":"", #線別
    "sLine2":"", #副線別
    "sTravel":"",
    "sWebarea":"", #區域
    "sKeyWord":"", #關鍵字
    "sFdate":"", #出發日期(起)
    "sTdate":"20160403", #出發日期(迄)
    "sDays":"0,99", #旅遊天數
    "sAmt":"",
    "sPsubject":"",
    "sProd":"",
    "sProdcomp":"",
    "sOrder":"date",
    "sTravelArea":"",
    "exa":"",
    "sFromAction":"NEW",
    "sProdGo":"0", #只找保證出團
    "sPageIndex":1, #第幾頁
    "sPageSize":99, #每頁幾筆資料
    "sWeekday":[], #星期
    "sPriceRange":[], #售價範圍
    "sTravelKind":[],
    "sUnPrice":"",
    "sTemplate":"",
    "sAgentOnly":""    
}

payload['sFdate'] = datetime.now().strftime('%Y%m%d')
# payload['sTdate'] = datetime(datetime.now().year, 12, 31).strftime('%Y%m%d')

#https://www.liontravel.com/WebSearch/Scripts/Array/TRS1NEW.js
#https://www.liontravel.com/WebSearch/Scripts/Array/TRS1NEWDOM.js

line1_key = []
line1_value = []
line2_key = []
line2_value = []
area_key = []
area_value = []

r = requests.get('https://www.liontravel.com/WebSearch/Scripts/Array/TRS1NEW.js')
city_dict  = eval(bs(r.text, 'lxml').text.split(';')[0].strip().split('=')[1].strip())
line1_dict  = eval(bs(r.text, 'lxml').text.split(';')[1].strip().split('=')[1].strip())
line2_dict = eval(bs(r.text, 'lxml').text.split(';')[2].strip().split('=')[1].strip())
area_dict = eval(bs(r.text, 'lxml').text.split(';')[3].strip().split('=')[1].strip())

for key, value in sorted(line1_dict.items()): 
    line1_key.append(key.replace('_',''))
    line1_value.append(value) 
for i in sorted(line1_dict.keys()):
    for key, value in sorted(line2_dict[i].items()):
        if key.replace('_','') == '':
            pass
        else:
            line2_key.append(key.replace('_',''))
            line2_value.append(value) 
for i in sorted(line1_dict.keys()):
    for j in sorted(line2_dict[i].keys()):
        if j == "_":
            pass
        else:
            for key, value in sorted(area_dict[j].items()): 
                if key.replace('_','') == '':
                    pass
                else:
                    area_key.append(key.replace('_',''))
                    area_value.append(value)
line1 = pd.DataFrame(line1_value, index=line1_key, columns=['線別'])
line2 = pd.DataFrame(line2_value, index=line2_key, columns=['副線別'])
area = pd.DataFrame(area_value, index=area_key, columns=['區域別'])

In [365]:
lionprod_con = []
for i in line1_key:
    for j in line2_key:
        for k in area_key:
            payload['sLine1'] = i
            payload['sLine2'] = j
            payload['sWebarea'] = k
            r = requests.post('https://www.liontravel.com/webpd/webpdse01.aspx/getdata', json = payload, headers = headers)
            pages = json.loads(json.loads(r.text)['d'])['PageInfo']['TotalPage']
            
            for page in range(pages):
                r = requests.post('https://www.liontravel.com/webpd/webpdse01.aspx/getdata', json = payload, headers = headers)
                js = json.loads(json.loads(r.text)['d'])['Grid']
                lionprod = pd.read_json(json.dumps(js))
                lionprod['Produrl'] = "na"
                for i in range(len(js)):
                    try:
                        lionprod['ProdAmt'].ix[i] = int(bs(js[i]['ProdAmt'], 'lxml').select('a')[0].text.strip('$').replace(',', ''))
                    except:
                        lionprod['ProdAmt'].ix[i] = 0
                    try:
                        lionprod['ProdCname'].ix[i] = bs(js[i]['ProdCname'], 'lxml').select('a')[0].text
                    except:
                        lionprod['ProdCname'].ix[i] = js[i]['ProdCname']
                    try:
                        lionprod['Produrl'].ix[i] = 'https://www.liontravel.com/webpd/webpdsh00.aspx?sProd={}'.format(lionprod['Prod'])
                    except:
                        pass

                lionprod['DataDate'] = datetime.now()
                lionprod['sLine1'] = payload['sLine1']
                lionprod['sLine2'] = payload['sLine2']
                lionprod['sWebarea'] = payload['sWebarea']
                lionprod_con.append(lionprod[['Prod', 'ProdD8', 'WeekDay', 'ProdCname', 'Produrl','ProdDays', \
                                              'sLine1', 'sLine2', 'sWebarea', 'ProdAmt', 'ProdMax', 'VtotSale', 'ProdGo', 'DataDate']])
                payload['sPageIndex'] += 1
                time.sleep(1.2)
lionprod_final = pd.concat(lionprod_con, ignore_index=True)
lionprod_final

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))

In [21]:
lionprod_final.to_csv("LionProd-"+datetime.now().strftime('%Y%m%d')+".csv")